In [1]:
from queue import Queue
from threading import Thread
import sys
import time
import cv2

class VideoReader:
    def __init__(self, video_file, queue_size=128):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(video_file)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)
        print('reading video from ', video_file)
        


    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        time.sleep(1)
        print('starting video reader...')
        return self

    def update(self):
        # keep looping infinitely
#         frame_num = 0
        while True:
#             if frame_num == 100:
#                 break
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                (grabbed, frame) = self.stream.read()
#                 frame_num += 1
                # add the frame to the queue
                if not grabbed:
                    print('no more frames')
                    self.stop()
                    return
                
                self.Q.put(frame)
                
                

    def read(self):
        # return next frame in the queue
        return self.Q.get()


    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True
        self.stream.release()
        print('stopped')
    
    def is_stopped(self):
        return self.stopped
    
    def is_full(self):
        return self.Q.full()
        
        
    def close(self):
        self.stop()

In [2]:
import os 
import glob
import cv2
import numpy as np


process_folder = True
process_file = False
save_frames = False
process_every_n_frames = 1
calc_drone_movement = True
skip_existing_folders = False

# Set max_extracted_frame to false to run on whole video
max_extracted_frame = False

observation_name = 'observation088'

if process_folder:
    parent_output_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-frames/raw-footage/' + observation_name
    video_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/' + observation_name
    video_files = glob.glob(video_folder + '*/*.MOV')
    existing_image_folders = glob.glob(parent_output_folder + '*/*')
    existing_folder_names = [os.path.basename(image_folder) for image_folder in existing_image_folders]
    print(video_folder)

if process_file:
    video_name = 'APR08_2018_C_DJI_0009' 
    animal_type = 'GZ'
    parent_output_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-frames/raw-footage-calibrate/'
    video_file = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/*/' + video_name + '.MOV'
    drone_movement_file = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/' + animal_type + '/' + video_name + '/localizations/drone_movement.npy' 
    video_files = glob.glob(video_file)
    existing_image_folders = glob.glob(parent_output_folder + '*/*')
    existing_folder_names = [os.path.basename(image_folder) for image_folder in existing_image_folders]
    
if process_folder:
    if not os.path.exists(video_folder):
        raise ValueError('The video folder does not exist.') 

print(len(video_files), 'video files found.')
print(len(existing_image_folders), 'existing image folders found')

/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088
12 video files found.
12 existing image folders found


In [3]:
video_files.sort()

print(len(video_files))

12


In [4]:
print(len(video_files))
print(len(existing_folder_names))

12
12


In [5]:
video_files = video_files[:]
print(video_files)

['/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0005.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0006.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0007.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0008.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_B_DJI_0005.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_B_DJI_0006.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_B_DJI_0007.MOV', '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR

In [6]:
if calc_drone_movement:
    warps = [np.eye(3)]
    total_warp = np.eye(3)
    
    
    
def calculate_transform(im1, im2, past_warp):

    warp_matrix = cv2.estimateRigidTransform(im1, im2, fullAffine=False) 

    warp_matrix = np.vstack((warp_matrix, np.array([0,0,1])))
    combined_warp = np.matmul(past_warp, warp_matrix)
    return warp_matrix, combined_warp

first_frame_list = [6246, 0, 0, 0, 7624, 0, 0, 0, 11228, 0, 0, 0]

for video_count, video_file in enumerate(video_files): 
    video_name = os.path.basename(video_file)
    video_name = video_name.split('.')[-2]
    if process_folder:
        if skip_existing_folders:
            if video_name in existing_folder_names:
                continue

        print('Processing', video_name)
    
    if save_frames:
        animal_type = video_file.split('/')[-2]
        output_folder = os.path.join(parent_output_folder, animal_type, video_name)
    
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    
        print('saving images to ', output_folder)
    else:
        print('frames are not being saved')
    




    frame_count = 0
    video_stream = VideoReader(video_file).start()
    time.sleep(1)
    last_frame = None
    
    
    while video_stream.more():
    

        if video_stream.is_full() or video_stream.is_stopped():
            if frame_count % 4000 == 0:
                print(frame_count, 'frames processed')
            if max_extracted_frame:
                if frame_count >= max_extracted_frame:
                    break




            frame = video_stream.read()

            if save_frames:
                if frame_count % process_every_n_frames == 0:
                    # make this the right number format
                    frame_name = "{}_{:05d}".format(os.path.splitext(video_name)[0], frame_count)
#                     frame_name = os.path.splitext(video_name)[0] + '_' + str(frame_count)
                    frame_file = os.path.join(output_folder, frame_name + '.jpg')
                    cv2.imwrite(frame_file, frame)

            if calc_drone_movement:
                if frame_count >= first_frame_list[video_count]:
                    if frame_count % 1 == 0:
                        if not last_frame is None:
                            warp, total_warp = calculate_transform(last_frame, frame, total_warp)

                            warps.append(warp)

                        last_frame = np.copy(frame)

            frame_count += 1

#         if frame_count == 9540:
#             break

    video_stream.stop()

Processing APR08_2018_A_DJI_0005
frames are not being saved
reading video from  /media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0005.MOV
starting video reader...
0 frames processed
4000 frames processed
8000 frames processed
12000 frames processed
16000 frames processed
no more frames
stopped
stopped
Processing APR08_2018_A_DJI_0006
frames are not being saved
reading video from  /media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0006.MOV
starting video reader...
0 frames processed
4000 frames processed
8000 frames processed
12000 frames processed
16000 frames processed
no more frames
stopped
stopped
Processing APR08_2018_A_DJI_0007
frames are not being saved
reading video from  /media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0007.MOV
starting video reader...
0 frames processed
4000 frames

In [ ]:
import math
a = 15
print(math.sqrt(a))

In [3]:
video_stream.stop()

stopping


In [7]:
video_name = 'NOV10_2017_DJI_0033' 
animal_type = 'PZ'

# optional
video_file = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/scare-clips/' + animal_type + '/' + video_name + '.MOV'
warps_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/raw-footage/observation088'
drone_movement_file = warps_folder +  '/' + 'drone_movement.npy' 
print(drone_movement_file)
np.save(drone_movement_file, warps)


/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/raw-footage/observation088/drone_movement.npy


In [8]:
print(drone_movement_file)

/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-frames/raw-footage/observation088/drone_movement.npy


In [7]:
len(warps)

228259

In [2]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 3.4.1 =====================================
  Version control:               3.4.1

  Platform:
    Timestamp:                   2018-05-19T13:50:22Z
    Host:                        Linux 4.4.0-101-generic x86_64
    CMake:                       3.9.0
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (3 files):          + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (1 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (5 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (9 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 FMA3 AVX AVX2

  C/C++:


In [4]:
print("num {:05d}".format(123))

num 00123
